In [1]:
%load_ext autoreload
%autoreload 2

In [73]:
from nep_lms.embeddings.variants import MiniLML6V3Variant

variant = MiniLML6V3Variant()
variant.train(epochs=10, lr=2e-5, early_stop=True, early_stop_patience=4, push_to_hub=False)

README.md: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss,Validation Loss,Nepali Triplets Loss
100,0.069600,No log,0.048725
200,0.029900,No log,0.043339


In [85]:
# variant.push_to_hub()

In [74]:
from sentence_transformers import SentenceTransformer
nep_mini_lm_v2 = SentenceTransformer("jangedoo/all-MiniLM-L6-v2-nepali")
nep_mini_lm_v3 = SentenceTransformer("jangedoo/all-MiniLM-L6-v3-nepali")
e5_small = SentenceTransformer("intfloat/multilingual-e5-small")

In [75]:
exp = variant.experiment
all_models = {
    'nep_mini_lm_v2': nep_mini_lm_v2,
    'e5_small': e5_small,
    'nep_mini_lm_v3': nep_mini_lm_v3,
    'nep_mini_lm_v3.1': variant.model,
}
exp.compare(models=all_models)
exp.plot_model_comparison()

In [76]:
import easyknn

texts = exp.nepali_news_ds['valid']['excerpt']
v2_embeddings = nep_mini_lm_v2.encode(texts, convert_to_numpy=True)
v3_embeddings = variant.model.encode(texts, convert_to_numpy=True)
e5_embeddings = e5_small.encode(texts, convert_to_numpy=True)

def get_knn(embeddings, texts):
    builder = easyknn.EmbeddingsIndexBuilder()
    builder.add(embeddings=embeddings, item_keys=range(len(texts)), items=texts)
    return easyknn.EasyKNN.from_builder_with_sklearn(builder=builder)

v2_knn = get_knn(v2_embeddings, texts)
v3_knn = get_knn(v3_embeddings, texts)
e5_knn = get_knn(e5_embeddings, texts)

In [84]:
import ipywidgets as w

item_idx = 33
v2_results, v2_distances = v2_knn.neighbors_by_item(item=item_idx, k=4)
v3_results, v3_distances = v3_knn.neighbors_by_item(item=item_idx, k=4)
e5_results, e5_distances = e5_knn.neighbors_by_item(item=item_idx, k=4)

w.HBox([
    w.VBox([w.HTML("v2<br/><ul>" + "\n".join([f"<li>{txt} ({dist:.2f})</li>" for txt, dist in zip(v2_results, v2_distances)]) + "</ul>")]),
    w.VBox([w.HTML("v3<br/><ul>" + "\n".join([f"<li>{txt} ({dist:.2f})</li>" for txt, dist in zip(v3_results, v3_distances)]) + "</ul>")]),
    w.VBox([w.HTML("e5<br/><ul>" + "\n".join([f"<li>{txt} ({dist:.2f})</li>" for txt, dist in zip(e5_results, e5_distances)]) + "</ul>")]),
])